In [ ]:
import pandas as pd

In [ ]:
# read weather data from: https://climat.meteo.gc.ca/historical_data/search_historic_data_f.html?searchType=stnName&timeframe=1&txtStationName=montreal&searchMethod=contains&optLimit=yearRange&StartYear=1840&EndYear=2024&Year=2024&Month=5&Day=1&selRowPerPage=25#stnNameTab
# execute this bash script to download data:

#!/bin/bash

station_ID=51157
start_year=2020
end_year=2024
path="weather_data"

for year in $(seq $start_year $end_year); do
    for month in $(seq 1 12); do
        wget -P ${path} --content-disposition "https://climate.weather.gc.ca/climate_data/bulk_data_e.html?format=csv&stationID=${station_ID}&Year=${year}&Month=${month}&Day=14&timeframe=1&submit=Download+Data"
    done
done

# path is tbe path to store weather files
# stationID is chosen from station inventory
# StationID used for project:

# MONTREAL INTL A: 51157
# OTTAWA GATINEAU A: 50719
# QUEBEC INTL A: 51457
# SHERBROOKE: 48371


In [ ]:
# merge files

months = ["01", "02", '03', '04', '05', '06', '07', '08', '09', '10', '11', '12']
years = ['2018', '2019', '2020', '2021', '2022', '2023']
dataframes = []
for year in years:
    for month in months:
        data = pd.read_csv("weather_data/en_climate_hourly_QC_7025251_" + month+ "-" + year + "_P1H.csv")
        dataframes.append(data)

data = pd.read_csv("weather_data/en_climate_hourly_QC_7025251_01-2024" + "_P1H.csv")
dataframes.append(data)

data_qc = pd.concat(dataframes)
data_qc.set_index("Date/Time (LST)", inplace=True)
data_qc.to_csv("montreal_intl_a_weather.csv")

In [ ]:
# read data
montreal_weather = pd.read_csv("montreal_intl_a_weather.csv")

# map column name
column_mapping = {
    "Temp (°C)": "temp",
    "Dew Point Temp (°C)": "rel_hum",
    "Wind Chill": "wind_chill",
    "Wind Spd (km/h)": "wind_speed",
    "Wind Dir (10s deg)": "wind_dir",
    "Weather": "condition",
    "Date/Time (LST)": "datetime"
}

montreal_weather = montreal_weather.rename(columns=column_mapping)

df = pd.DataFrame(montreal_weather)

In [ ]:
df.tail()